# Explore models
Try SOTA text summarization models for their performance on speech/lecture transcripts
Since Sat. Oct. 23rd, 2021


In [ ]:
## Setup
import torch
# import tensorflow
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import PegasusTokenizer, BigBirdPegasusForConditionalGeneration, BigBirdPegasusConfig
# import protobuf

from util import *


txt_ted = get_ted_eg('Cuddy')['transcript']
# print(txt_ted)
ic(len(txt_ted.split()))
txt_498 = get_498_eg()
txt_498_sec = get_498_eg(section=True)
# print(txt_498[:400])
ic(len(txt_498.split()))
ic(len(txt_498_sec.split()))



## BigBird
"Big Bird: Transformers for Longer Sequences".
Manzil Zaheer, Guru Guruganesh, Kumar Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, Amr Ahmed.
*NeurIPS 2020*.



In [2]:
def bigbird(text):
    """
    `BigBird` in torch

    Looks like doesn't support summarization
    """
    summarizer = pipeline(
        'summarization',
        model='google/bigbird-roberta-base',
        tokenizer='google/bigbird-roberta-base'
    )
    summarizer(text, min_length=5, max_length=20)
# bigbird('hello world')


def bigbird_pegasus(text):
    model = BigBirdPegasusForConditionalGeneration.from_pretrained('google/bigbird-pegasus-large-arxiv')
    tokenizer = PegasusTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')
    inputs = tokenizer([text], max_length=4096, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=256, early_stopping=True)
    return tokenizer.batch_decode(summary_ids)
"""
Hyper-parameter seems to affect the result drastically

On TED.com `Do schools kill creativity?`
['<s> we present a brief discussion of the nature of education in the era of '
'big data and big rip.<n> we start with a brief history of education in the '
'era of big rip.<n> we then turn to a brief discussion of the nature of '
'education in the era of big rip.<n> we conclude with a discussion of the '
'future of education.']

['<s> this is an elementary introduction to the notion of the distance between '
'two points on the unit circle.<n> the distance is defined as the difference '
'between the value of the unit circle and the value of the point on the unit '
'circle.<n> the definition of the distance is based on the concept of an '
'equivalence between the value of the unit circle and the value of the point '
'on the unit circle.<n> the distance is defined as the difference between the '
'value of the unit circle and the value of the point on the unit circle.<n> '
'the definition of the distance is based on the concept of an equivalence '
'between the value of the unit circle and the value of the point on the unit '
'circle. <n> [ [ section ] ] this is an elementary introduction to the notion '
'of the distance between two points on the unit circle.<n> the distance is '
'defined as the difference between the value of the unit circle and the value '
'of the point on the unit circle.<n> the definition of the distance is based '
'on the concept of an equivalence between the value of the unit circle and '
'the value of the point on the unit circle.<n> the distance is defined as the '
'difference between the value of the unit circle and the value of the point '
'on the unit circle ']

On TED.com `Your body language may shape who you are`
['<s> a lot of us have found ourselves in situations where we are not sure '
'what we are doing, where we are not sure what to do, where we are not sure '
'what to say, where we are not sure what to do, where we are not sure what to '
'say, where we are not confident, where we are not sure what to do, where we '
'are not sure what to do, where we are not confident, where we are not sure '
'what to do, where we are not sure what to do, where we are not sure what to '
'do, where we are not confident, where we are not sure what to do, where we '
'are not sure what to do, where we are not confident, where we are not sure '
'what to do, where we are not confident, where we are not sure what to do, '
'where we are not confident, where we are not confident, where we are not '
'sure what to do, where we are not confident, where we are not confident, '
'where we are not confident, where we are not confident, where we are not '
'confident, where we are not confident, where we are not']
"""



# txt_498 = get_1st_n_words(txt_498)
# ic(txt_498)
ic(bigbird_pegasus(txt_ted))
# ic(bigbird_pegasus(txt_498))




ic| txt_498[:400]: ("No, you don't need a reason. Apple TV. All right. Apple TV. Sweet don't have "
                    "to carry around a dongle. Welcome, welcome everybody. Yeah, it's um the "
                    "gloomy day. I got a bunch of great emails saying I can't make it. I feel it. "
                    "I feel it. But anyway, let's just jump right into how many people have "
                    "groups? Awesome. That's all you left. The scene. Pretty much everybody. So "
                    "we'll will tease out who mad")
ic| bigbird_pegasus(txt_ted): ['<s> a lot of us have found ourselves in situations where we are not sure '
                               'what we are doing, where we are not sure what to do, where we are not sure '
                               'what to say, where we are not sure what to do, where we are not sure what to '
                               'say, where we are not confident, where we are not sure what to do, where we '
                        

['<s> a lot of us have found ourselves in situations where we are not sure what we are doing, where we are not sure what to do, where we are not sure what to say, where we are not sure what to do, where we are not sure what to say, where we are not confident, where we are not sure what to do, where we are not sure what to do, where we are not confident, where we are not sure what to do, where we are not sure what to do, where we are not sure what to do, where we are not confident, where we are not sure what to do, where we are not sure what to do, where we are not confident, where we are not sure what to do, where we are not confident, where we are not sure what to do, where we are not confident, where we are not confident, where we are not sure what to do, where we are not confident, where we are not confident, where we are not confident, where we are not confident, where we are not confident, where we are not confident, where we are not']

## T5



In [8]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")


def summarize(text):
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=4096,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]


ic(summarize(txt_ted))



Token indices sequence length is longer than the specified maximum sequence length for this model (4871 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

## Distill BART
Token size of 1024 is not large enough



In [4]:
# # summarizer = pipeline('summarization', model='sshleifer/distilbart-cnn-12-6')
# summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
# ic(summarizer(txt_ted, max_length=300))


Token indices sequence length is longer than the specified maximum sequence length for this model (4591 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

## Pegasus Xsum
Max token of 512



In [6]:
# summarizer = pipeline('summarization', model='google/pegasus-xsum')
# ic(summarizer(txt_ted, max_length=256))



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4572 > 512). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

## BART on SAMSum
1024 tokens maximum



In [7]:
# summarizer = pipeline('summarization', model='philschmid/bart-large-cnn-samsum')
# ic(summarizer(txt_ted, max_length=256))



Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4591 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

## Pegasus
Max token 1024



In [9]:
summarizer = pipeline('summarization', model='google/pegasus-large')
ic(summarizer(txt_ted, max_length=256))




Downloading:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4571 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self